In [1]:
print("hello World!")

hello World!


In [2]:
%pwd

'/workspaces/End-to-End-Medical-Chatbot-AI-2-/research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'/workspaces/End-to-End-Medical-Chatbot-AI-2-'

In [4]:
import sys, pkgutil
print(sys.executable)
print("langchain" in {m.name for m in pkgutil.iter_modules()})


/opt/conda/envs/medbot/bin/python
False


In [ ]:
from langchain.document_loaders import PyPDFLoader , DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


/opt/conda/envs/medbot/bin/python


In [1]:
# Extarction of data from the PDF file
def load_pdf_file(data):
    loader = DirectoryLoader(data,
                             glob = "*.pdf",
                             loader_cls = PyPDFLoader)
    documents = loader.load()

    return documents